<a href="https://colab.research.google.com/github/sathya981/CovidBot/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
path = "/content/gdrive/My Drive/model" 



In [3]:
import json
import os
import nltk
import numpy as np
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import random
import pickle
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import files
files.upload()


Saving intents.json to intents.json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],\n         "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],\n         "context": [""]\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],\n         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],\n         "context": [""]\n        },\n\n \t{"tag": "rate",\n         "patterns": ["What is death rate or fatality rate of the disease?","What is death rate or fatality rate of coronavirus?"],\n         "responses": ["The fatality rate of covid-19 is less than 3%"],\n         "context": [""]\n        },\n\t{"tag": "pet",\n         "patterns": ["Can my pet animal spread the disease?"," Can cats and dogs spread the disease?"],\n         "responses": ["No evidence ex

In [5]:
data=open('intents.json').read()
intents = json.loads(data)



In [6]:
vocab=[]
sentence=[]
label=[]

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_tokens=nltk.word_tokenize(pattern)
        vocab.extend(word_tokens)
        sentence.append((word_tokens,intent['tag']))
    label.append(intent['tag'])


In [7]:
ignore_words=['?','!']

vocab = [stemmer.stem(w.lower()) for w in vocab if w not in ignore_words]
vocab = sorted(list(set(vocab)))
# sort classes
label = sorted(list(set(label)))

In [8]:


f =open("/content/gdrive/My Drive/model/vocab.pkl", "wb") 
pickle.dump(vocab,f)
f.close()

file= open("/content/gdrive/My Drive/model/label.pkl", "wb") 
pickle.dump(label,file)
file.close()




training_data=[]
for statement in sentence:
    features=[stemmer.stem(w.lower()) for w in statement[0] ]
    target=statement[1]
    
    bag=[0]*len(vocab)
    
    for w in vocab:
        if w in features:
            bag.append(1)
        else:
            bag.append(0)
            
    target_no=label.index(target)
    target_row=[0]*len(label)
    target_row[target_no]=1
    training_data.append((bag,target_row))
print(np.shape(bag))

(290,)


In [9]:
random.shuffle(training_data)
training_data=np.array(training_data)
train_x=list(training_data[:,0])
print(np.shape(train_x))
train_y=list(training_data[:,1])

(60, 290)


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


from keras.optimizers import adam


model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#fitting and saving the model 
model.fit(np.array(train_x), np.array(train_y), epochs=250, batch_size=5, verbose=1)


Using TensorFlow backend.


Epoch 1/250
60/60 [==============================] - 2s 30ms/step - loss: 2.8037 - accuracy: 0.0667
Epoch 2/250
60/60 [==============================] - 0s 580us/step - loss: 2.7838 - accuracy: 0.1333
Epoch 3/250
60/60 [==============================] - 0s 567us/step - loss: 2.7661 - accuracy: 0.1000
Epoch 4/250
60/60 [==============================] - 0s 573us/step - loss: 2.6917 - accuracy: 0.1500
Epoch 5/250
60/60 [==============================] - 0s 549us/step - loss: 2.6721 - accuracy: 0.1833
Epoch 6/250
60/60 [==============================] - 0s 564us/step - loss: 2.6679 - accuracy: 0.1500
Epoch 7/250
60/60 [==============================] - 0s 559us/step - loss: 2.6315 - accuracy: 0.1667
Epoch 8/250
60/60 [==============================] - 0s 566us/step - loss: 2.5716 - accuracy: 0.2333
Epoch 9/250
60/60 [==============================] - 0s 586us/step - loss: 2.4757 - accuracy: 0.3500
Epoch 10/250
60/60 [==============================] - 0s 570us/step - loss: 2.4940 - accurac

In [11]:
model.save(os.path.join(path,"chatbot.h5"))
